# Goal
- Train GPT2 on wiki text

## Steps
- Read, download data
- Train tokenizer
- Prepare sliding window data loader
- Use GPT2 model
- Use train/test loop

### Read, download data

In [1]:
raw_text = ["abcd" * 10]*10
train_dataset = {
    "text": raw_text
}
test_dataset = {
    "text": raw_text
}
val_dataset = {
    "text": raw_text
}

# convert train_dataset, test_dataset, val_dataset to huggingface datasets
from datasets import Dataset
train_dataset = Dataset.from_dict(train_dataset)
test_dataset = Dataset.from_dict(test_dataset)
val_dataset = Dataset.from_dict(val_dataset)

train_dataset

Dataset({
    features: ['text'],
    num_rows: 10
})

In [2]:
train_dataset["text"][0]

'abcdabcdabcdabcdabcdabcdabcdabcdabcdabcd'

### Train tokenizer

In [3]:
import tokenizers
import transformers
import tiktoken

# get gpt2 tokenizer
wrapped_tokenizer = transformers.GPT2TokenizerFast.from_pretrained("gpt2", padding_side="left")

# set padding token
wrapped_tokenizer.pad_token = wrapped_tokenizer.eos_token



In [ ]:
wrapped_tokenizer.tokenize("abcdabcd")

### Prepare sliding window data loader

In [4]:
import torch 

def slide_window(text_batch):
    text_batch['input_words'] = []
    text_batch['output_words'] = []
    text_batch['input_ids_raw'] = []
    text_batch['output_ids_raw'] = []

    text_batch['input_ids'] = []
    text_batch['output_ids'] = []
    text_batch['attention_mask'] = []

    for text in text_batch['text']:
        
        tokens = wrapped_tokenizer.tokenize(text)
        tokens.append(wrapped_tokenizer.eos_token)  # add eos token to the end of the tokens
        input_tokens = tokens[:-1]
        output_tokens = tokens[1:]
        
        text_batch['input_words'].append(input_tokens)
        text_batch['output_words'].append(output_tokens)

        input_ids_raw = wrapped_tokenizer.convert_tokens_to_ids(input_tokens)
        output_ids_raw = wrapped_tokenizer.convert_tokens_to_ids(output_tokens)

        text_batch['input_ids_raw'].append(input_ids_raw)
        text_batch['output_ids_raw'].append(output_ids_raw)

        # pad, truncate, and convert to tensor
        input_ids = wrapped_tokenizer.pad({"input_ids": input_ids_raw}, padding="max_length", max_length=100, return_tensors="pt")["input_ids"]
        output_ids = wrapped_tokenizer.pad({"input_ids": output_ids_raw}, padding="max_length", max_length=100, return_tensors="pt")["input_ids"]
        attention_mask = [0] * (100 - len(input_ids_raw)) + [1] * len(input_ids_raw)

        assert len(attention_mask) == 100
        text_batch['input_ids'].append(input_ids)
        text_batch['output_ids'].append(output_ids)
        text_batch['attention_mask'].append(attention_mask)

    return text_batch 

tokenized_train_dataset = train_dataset.map(slide_window, batched=True)
tokenized_val_dataset = val_dataset.map(slide_window, batched=True)
tokenized_test_dataset = test_dataset.map(slide_window, batched=True)

tokenized_train_dataset


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_words', 'output_words', 'input_ids_raw', 'output_ids_raw', 'input_ids', 'output_ids', 'attention_mask'],
    num_rows: 10
})

In [5]:
tokenized_train_dataset['input_words'][0], tokenized_train_dataset['output_words'][0]

(['ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd'],
 ['cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  '<|endoftext|>'])

In [6]:
import torch 
from datasets import Dataset as HFDataset
from torch.utils.data import Dataset

class HuggingFaceDataset(Dataset):
    """
    Wraps a Hugging Face Dataset to be used with a PyTorch DataLoader.

    Assumes the Hugging Face dataset has 'input' and 'target' columns.
    """

    def __init__(self, hf_dataset: HFDataset):
        self.hf_dataset = hf_dataset

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        item = self.hf_dataset[idx]
        return item['input_ids'], item['output_ids'], item['attention_mask']

def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    output_ids = [item[1] for item in batch]
    attention_mask = [item[2] for item in batch]
    input_ids_list = torch.tensor(input_ids)
    output_ids_list = torch.tensor(output_ids)
    attention_mask_list = torch.tensor(attention_mask)
    return input_ids_list, output_ids_list, attention_mask_list

batch_size = 20
train_torch_dataset = HuggingFaceDataset(tokenized_train_dataset)
val_torch_dataset = HuggingFaceDataset(tokenized_val_dataset)
test_torch_dataset = HuggingFaceDataset(tokenized_test_dataset)

train_torch_dataloader = torch.utils.data.DataLoader(
    train_torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
val_torch_dataloader = torch.utils.data.DataLoader(
    val_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)
test_torch_dataloader = torch.utils.data.DataLoader(
    test_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

train_torch_dataloader

In [7]:
batch = next(iter(train_torch_dataloader)) # (input_ids, output_ids)
input_ids, output_ids, attention_masks = batch
input_ids.shape, output_ids.shape, attention_masks.shape

(torch.Size([10, 100]), torch.Size([10, 100]), torch.Size([10, 100]))

### Use GPT2 model

In [8]:
from models import GPT2

num_train_batches = tokenized_train_dataset.num_rows // batch_size

config = {
        "emb_dim": 128,
        "heads": 2,
        "layers": 2,
        "vocab_size": 50257,
        "context_length": 128,
        "device": torch.device("cuda:0"),
        "drop_out": 0.1,
        "train_test_split": 0.8,
        "num_epochs": 50,
        "model_path": "../model_files/gpt2_emotion.pth",
        "num_train_batches" : num_train_batches
    }

gpt2 = GPT2(config)
gpt2.to(config['device'])
gpt2

GPT2(
  (token_embedding): Embedding(50257, 128)
  (position_embedding): Embedding(128, 128)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (layer_norm1): LayerNorm()
      (layer_norm2): LayerNorm()
      (self_attention_block): MultiHeadAttention(
        (W_Q): Linear(in_features=128, out_features=128, bias=True)
        (W_K): Linear(in_features=128, out_features=128, bias=True)
        (W_V): Linear(in_features=128, out_features=128, bias=True)
        (out_project): Linear(in_features=128, out_features=128, bias=True)
      )
      (feed_forward): FeedForward(
        (feed_forward): Sequential(
          (0): Linear(in_features=128, out_features=512, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=512, out_features=128, bias=True)
        )
      )
    )
    (1): TransformerBlock(
      (layer_norm1): LayerNorm()
      (layer_norm2): LayerNorm()
      (self_attention_block): MultiHeadAttention(
        (W_Q): Linear(in_f

### Use train/test loop

In [9]:
from utils import train

train(gpt2, train_torch_dataloader, val_torch_dataloader, config)
torch.save(gpt2.state_dict(), config["model_path"]) # Save the model

torch.Size([])
At epoch 1 batch 1 of num_batches 0Average batch loss: 10.932890892028809
Test loss without mask: at epoch 0 10.892751693725586 Test perplexity without mask: 53785.09765625
torch.Size([])
At epoch 2 batch 1 of num_batches 0Average batch loss: 10.892603874206543
Test loss without mask: at epoch 1 10.853691101074219 Test perplexity without mask: 51724.71875
torch.Size([])
At epoch 3 batch 1 of num_batches 0Average batch loss: 10.853998184204102
Test loss without mask: at epoch 2 10.812539100646973 Test perplexity without mask: 49639.34765625
torch.Size([])
At epoch 4 batch 1 of num_batches 0Average batch loss: 10.813957214355469
Test loss without mask: at epoch 3 10.77685260772705 Test perplexity without mask: 47899.12890625
torch.Size([])
At epoch 5 batch 1 of num_batches 0Average batch loss: 10.775415420532227
Test loss without mask: at epoch 4 10.737105369567871 Test perplexity without mask: 46032.61328125
torch.Size([])
At epoch 6 batch 1 of num_batches 0Average batch 

In [ ]:
x = torch.randn(2, 5)
mask = torch.tensor([0, 1])
x[mask == 0] = 0
x

In [ ]:
# load GPT2 from config.model_path
import os 

if os.path.exists(config['model_path']):
    gpt2.load_state_dict(torch.load(config['model_path']))
    print("model loaded")



### Generate text

In [11]:
tokenized_train_dataset[1]

{'text': 'abcdabcdabcdabcdabcdabcdabcdabcdabcdabcd',
 'input_words': ['ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd'],
 'output_words': ['cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  'ab',
  'cd',
  '<|endoftext|>'],
 'input_ids_raw': [397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210],
 'output_ids_raw': [10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  397,
  10210,
  50256],
 'input_ids': [50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50

In [ ]:

x = tokenized_train_dataset[1]['input_ids']
y = tokenized_train_dataset[1]['output_ids']
mask = tokenized_train_dataset[1]['attention_mask']

x, y, mask = torch.tensor(x), torch.tensor(y), torch.tensor(mask)

x[mask==1], y[mask==1]

In [13]:
tokenized = wrapped_tokenizer("cd", truncation=True, max_length=128, padding="max_length", return_tensors="pt")


input_ids = tokenized['input_ids'].to(config["device"])
print(input_ids)

prediction = gpt2(input_ids)
#next_token_decoded = wrapped_tokenizer.decode(next_token)
#next_token_decoded
next_token_decoded = wrapped_tokenizer.decode(prediction[0, -1].argmax().item())
next_token_decoded

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 5

'ab'

In [14]:

def generate_text(starting_text, model, tokenizer, config, num_output_tokens=10):
    input_encoding = tokenizer(starting_text, return_tensors="pt")
    device = config["device"]
    output_tokens = []

    input_encoding = tokenizer(starting_text, truncation=True, max_length=100, padding="max_length", return_tensors="pt")
    input_ids = input_encoding['input_ids'].to(device)
    

    for i in range(num_output_tokens):
        
        next_token_logits = model(input_ids)[:,-1,:]
        next_token = next_token_logits.argmax(dim=-1)

        output_tokens.append(next_token.item())

        next_token = next_token.to(device)
        

        # Append the predicted token to the input for the next iteration
        input_ids = torch.cat((input_ids, next_token.unsqueeze(0)), dim=1)
        input_ids = input_ids[:, -128:]

        

        



    output_text = tokenizer.decode(output_tokens)
        #output_text += next_text
    print(f"{starting_text} -> {output_text}")

generate_text("ab", gpt2, wrapped_tokenizer, config)


ab -> cdabcdabcdabcdabcdab
